# Лабораторная работа №3. Методы поиска подстроки в строке.
## Выполнил студент группы БСТ1904 Комлев А.С.
## Задание 1

## Алгоритм Кнута-Морриса-Пратта

In [1]:
import time
import random
from tabulate import tabulate


In [2]:
def prefix(s):
    p = [0] * len(s)
    for i in range(1, len(s)):
        k = p[i - 1]
        while k > 0 and s[k] != s[i]:
            k = p[k - 1]
        if s[k] == s[i]:
            k += 1
        p[i] = k
    return p

In [3]:
def KMP(s, w):
    A = []
    k = 0
        
    p = prefix(s)

    for i in range(len(s)):
        while k > 0 and s[i] != w[k]:
            k = p[k-1]
        if s[i] == w[k]:
            k += 1
        if k == len(w):
            A.append((i - len(w) + 1, i+1))
            k = p[k-1]

    return A

In [4]:
print(KMP("aabaabaaaabaabaaab", "aabaa"))

[(0, 5), (3, 8), (8, 13), (11, 16)]


## Упрощенный Бойера-Мура

In [5]:
def preprocess(w):
    T = [len(w)]*256
    for i in range(len(w) - 1):
        T[ord(w[i])] = len(w) - 1 - i
    return T

In [6]:
def BM(s, w):
    A = []
    T = preprocess(w)
    skip = 0
    while len(s) - skip >= len(w):
        if s[skip:skip + len(w)] == w:
            A.append((skip, skip + len(w)))
        skip += T[ord(s[skip + len(w) - 1])]
    return A

In [7]:
print( BM("Hoola-Hoola girls like Hooligans", "Hooligan"))

[(23, 31)]


## Встроенный поиск

In [8]:
def builtin_search(s, w):
    A = []
    index = s.find(w)
    while index != -1:
        A.append((index, index + len(w)))
        index = s.find(w, index + 1)
    return A

## Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру.

In [9]:
def search(s, w, fn=KMP, ignore_case=True, ignore_space=False):
    """ Поиск подстроки с возможностью игнорирования пробелов и регистра """
    _s = s
    _w = w
    if ignore_case:
        _s = _s.lower()
        _w = _w.lower()
    
    if ignore_space:
        _s = _s.replace(' ', '')
        _w = _w.replace(' ', '')

    A = fn(_s, _w)

    if ignore_space:
        nonspace = 0
        kmp_without_space = 0
        index = 0
        while kmp_without_space < len(A) and index < len(s):
            if A[kmp_without_space][0] == nonspace:
                index_with_space_offsets = index
                chars_count = 0
                while chars_count < len(_w) and index_with_space_offsets < len(s):
                    if s[index_with_space_offsets] != ' ':
                        chars_count += 1
                    index_with_space_offsets += 1
                A[kmp_without_space] = (index, index_with_space_offsets)
                kmp_without_space += 1
            if s[index] != ' ': nonspace += 1
            index += 1
    
    return A

## Сравнение алгоритмов

In [10]:

bench_count = 10

time_consumed = {}
for alg in (KMP, BM, builtin_search):
    time_start = time.perf_counter()
    for i in range(bench_count):
        alg('ABCABCABCDABDABDABC', 'ABC')
        alg('ADBCHDBDBAHBCBACBDBEHABBBVHABBBVHABDAVHHCBABCB', 'AB')
    time_end = time.perf_counter()
    time_consumed[alg.__name__] = (time_end - time_start) / bench_count

sorted_time = sorted(time_consumed.items(), key=lambda kv: kv[1])
tabulate(sorted_time, headers=['Алгоритм','Время'], tablefmt='html', showindex="always")

,Алгоритм,Время
0,builtin_search,5.24e-06
1,KMP,3.191e-05
2,BM,3.484e-05


## Задание 2 «Пятнашки»

In [11]:
from queue import PriorityQueue

N = 4

# Движение пятнашек
def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

# Функция для определения есть решение или нет
def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2

# Класс позиции
class Position:
    # Конструктор класса, который принимает позицию и начальную дистанцию
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    # Метод, который срабатывает при сравнении (<) объектта с другим объектом
    def __lt__(self, other):
        return self.start_distance < other.start_distance

    # Метод, который срабатывает при использовании объекта как строки
    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

# Разгадка
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
# Загадка
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

# Смотрим, можно ли в данной расстановке найти решение
# Если нет, то сообщаем об этом
if parity(start) == 0:
    print('Нерешаемо')
# Иначе ищем этот путь
else:
    # Кортеж загадки
    start = tuple(start)
    
    # Первоначальная позиция
    p = Position(start, 0)
    print("Первоначальная позиция: " + "\n", p)
    print()
    
    # 1) Кладем в очередь с приоритетом первоначальную позицию
    candidates = PriorityQueue()
    candidates.put(p)

    # Кортеж посещенных позиций
    visited = set([p])

    # Откуда пришли
    came_from = {p.position: None}
    
    # Пока решение не найдено
    while p.position != SOLVED:
        # 2) Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        # 3) Кладем в очередь все соседние позиции
        # 4) Повторяем пункты 2-4 пока в пункте 2 не вытащим конечную позицию
        for k in moves(p.position):
            if k not in visited:
                # В candifates хранятся всевозможные позиции
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    # path - последовательное решение головоловки (путь)
    path = []
    # Сохраняем конечную позицию
    prev = p
    # Идем в обратном порядке и запоминаем очередность хода в path
    while p.position != start:
        # Запоминаем откуда ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print("Оптимальный путь к решению:" + "\n", path)

Первоначальная позиция: 
   1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

Оптимальный путь к решению:
 [15, 14, 10, 13, 9, 10, 14, 15]


# Вывод

В ходе выполнения данной лабораторной работы были изучены алгоритмы поиска таккие, как Алгорим Алгоритм Кнута-Морриса-Пратта, Упрощенный Бойера-Мура, Встроенный поиск Python и предусмотрена возможность существования пробела. Также была решена задача Пятнашки.